# 虚拟变量练习

在本练习中，你将从项目数据集中创建虚拟变量。其目的是将如下的类别变量进行转换：

| Project ID | Project Category |
|------------|------------------|
| 0          | Energy           |
| 1          | Transportation   |
| 2          | Health           |
| 3          | Employment       |

转换成如下的新特征：

| Project ID | Energy | Transportation | Health | Employment |
|------------|--------|----------------|--------|------------|
| 0          | 1      | 0              | 0      | 0          |
| 1          | 0      | 1              | 0      | 0          |
| 2          | 0      | 0              | 1      | 0          |
| 3          | 0      | 0              | 0      | 1          |


（注意：如果你要使用的模型会受到数据多重共线性的影响，你应该删掉一列，以避免冗余信息。） 

做数据转换的原因是机器学习算法接收的是数字而不是文本。文本需要被转换成数字。你可以将数字分配给每个类型，如1、2、3、4。但是类别变量没有内在的顺序，你需要在你的特征中表达这一特点。

使用 pandas 的 [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) 方法创建虚拟变量非常简单。在本练习中，你会对世界银行数据集创建虚拟变量；然而，有一个注意事项：世界银行数据不是非常干净，所以你需要先探索和处理数据。

你主要处理 sector 变量中的文本数据。

运行下方单元格中的代码，读入世界银行项目数据集，筛选出文本变量的数据。 

In [ ]:
import pandas as pd
import numpy as np

# read in the projects data set and do basic wrangling 
projects = pd.read_csv('../data/projects_data.csv', dtype=str)
projects.drop('Unnamed: 56', axis=1, inplace=True)
projects['totalamt'] = pd.to_numeric(projects['totalamt'].str.replace(',', ''))
projects['countryname'] = projects['countryname'].str.split(';', expand=True)[0]
projects['boardapprovaldate'] = pd.to_datetime(projects['boardapprovaldate'])

# keep the project name, lending, sector and theme data
sector = projects.copy()
sector = sector[['project_name', 'lendinginstr', 'sector1', 'sector2', 'sector3', 'sector4', 'sector5', 'sector',
          'mjsector1', 'mjsector2', 'mjsector3', 'mjsector4', 'mjsector5',
          'mjsector', 'theme1', 'theme2', 'theme3', 'theme4', 'theme5', 'theme ',
          'goal', 'financier', 'mjtheme1name', 'mjtheme2name', 'mjtheme3name',
          'mjtheme4name', 'mjtheme5name']]

运行下方单元格中的代码。会显示出每个变量空值的百分比。注意到变量 mjsector1 到 mjsector5 全是空值。从 mjtheme1name 到 mjtheme5name 和变量 theme 也全是空值。  

这些变量有这么多空值，所以我们猜测可能不是很重要。

In [ ]:
# output percentage of values that are missing
100 * sector.isnull().sum() / sector.shape[0]

# 练习 1

变量 sector1 看起来不错，它没有空值。在下一个单元格，保存去重的 sector1 的值到一个列表中，然后输出结果。你会用到 sort_values() 和 unique() 方法。

In [ ]:
# TODO: Create a list of the unique values in sector1. Use the sort_values() and unique() pandas methods. 
# And then convert those results into a Python list
uniquesectors1 = None
uniquesectors1

In [ ]:
# run this code cell to see the number of unique values
print('Number of unique values in sector1:', len(uniquesectors1))

3060 个类别确实很多。记得我们说过，对于虚拟变量，如果类别有 n 个值，你就需要新建 n-1 个虚拟变量！也就是说 3059 个新的列！ 

# 练习 2

变量 'sector1' 存在以下问题。首先，有些值是 '!$!0'。应该被替换为 NaN。

还有，每个 sector1 的值都以 10 位或 11 位的字符串结尾，如 '!$!49!$!EP'。一些字段 (sector) 在表中出现了两次：
 'Other Industry; Trade and Services!$!70!$!YZ',
 'Other Industry; Trade and Services!$!63!$!YZ',

但看起来他们其实是一个字段。你需要删掉 ‘!’ 之后的所有内容。 

很多变量 sector1 的值以 '(Historic)' 开头。试着删除这个短语。

### replace() 方法

pandas 的话，你可以使用 replace() 方法搜索这个字符串然后用另一个字符串替换它。如果你已经知道待查找字符串，replace() 方法很简明。例如，你想从下面的数据中删除字符串 '(Trial)'：

| data                     |
|--------------------------|
| '(Trial) Banking'        |
| 'Banking'                |
| 'Farming'                |
| '(Trial) Transportation' |

你可以使用 `df['data'].replace('(Trial'), '')` 来用空字串替换 (Trial)。

### 正则表达式
再来看看这个数据：

| data                                           |
|------------------------------------------------|
| 'Other Industry; Trade and Services?$ab' |
| 'Other Industry; Trade and Services?ceg' |

这样的数据处理起来有点难度。在本例中，你要删除的字符串有一种模式 (pattern)，就是以 '!' 开头，后接着若干长度的字符。当你想要匹配某种模式的字符串，你可以使用 [正则表达式](https://en.wikipedia.org/wiki/Regular_expression)。

replace 方法可以接收一个正则表达式。那么
df['data'].replace('?.+', regex=True) 中 '?.+' 表示找到一个字符集以 ‘?' 开始，后跟着一个或多个字符。你可以在这里找到 [正则表达式速查表](https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285)。

修正下方代码单元格中的错误。

In [ ]:
# TODO: In the sector1 variable, replace the string '!$10' with nan
#       Put the results back into the sector1 variable
# HINT: you can use the pandas replace() method and numpy.nan

# TODO: In the sector1 variable, remove the last 10 or 11 characters from the sector1 variable.
# HINT: There is more than one way to do this. To do it with one line of code,
# you can use the replace method with a regex expression '!.+'
# That regex expression looks for a string with an exclamation
# point followed by one or more characters

# TODO: Remove the string '(Historic)' from the sector1 variable
# HINT: You can use the replace method

print('Number of unique sectors after cleaning:', len(list(sector['sector1'].unique())))
print('Percentage of null values after cleaning:', 100 * sector['sector1'].isnull().sum() / sector['sector1'].shape[0])

现在有 156 个去重的类别值。比 3060 要好一些。如果你要用这样的数据来训练监督学习模型，你可以试着把这 156 个值转换成虚拟变量。你还得训练和测试模型来看它们是不是一个好的特征。

你可以试着将相同的类别变量整合到一起，这是第 4 部分 (part 4) 的挑战练习的内容。

还有一些行包含 NaN 值。要如何填充这些空值呢？ 

可需要从变量 'project_name' 或 'lendinginstr' 里先确定一个合适的类别。如果你创建的虚拟变量里包含 NaN ，那么你应该考虑新建一个变量，并全用 0 来表示 NaN。 或者就把这些记录从数据集中删掉。pandas 默认会忽视这些 NaN。意味着，对于给定的行，如果 sector1 的值是 NaN，所有的虚拟变量都取 0。

别忘了要结合数据集的实际含义！我们是为了机器学习算法准备这个数据。无论是使用什么方法构建新特征，你都需要使用这些数据训练模型。所以，如果你的新数据不包含 sector1 值，你还得在你的训练集中进行一些数据工程操作。

# 练习 3

在下一个练习中，使用 pandas 的 pd.get_dummies() 方法来创建虚拟变量。然后使用 concat() 方法将虚拟变量连接 (concatenate) 到含有项目 totalamt 变量和项目年份 boardapprovaldate 的 dataframe 中。

In [ ]:
# TODO: Create dummy variables from the sector1 data. Put the results into a dataframe called dummies
# Hint: Use the get_dummies method
dummies = None

# TODO: Create a new dataframe called df by 
#       filtering the projects data for the totalamt and
#       the year from boardapprovaldate
projects['year'] = None
df = None

# TODO: Concatenate the results of dummies and projects
#       into a single data frame
df_final = None

df_final.head()

你可以继续用其他方法整合 sector 变量。比如，在下个练习中，你会发现相似的类别，要把它们整合起来。 

记得，整合的程度取决于你的机器学习模型的性能，以及你的硬件是否能处理内存中更多的特征。如果你的硬件可以支持处理 3060 个新特征，并且你的机器学习算法表现更好，那就很好！

# 练习 4（困难）

但是，你还有什么其他处理变量 sector1 的方法吗？

变量 'sector1' 现在的空值率为 3.49% 。似乎和 'sector' 一列的空值数量是一样的。如果你向下滚动到计算每个变量空值的百分比的地方就会发现这个现象。 

也许 'sector1' 和 'sector' 变量包含的是一样的信息。如果你查看 'sector' 变量，它同样需要清洗。其值看起来像是这样：

'Urban Transport;Urban Transport;Public Administration - Transportation'

看起来 'sector' 变量将从变量 'sector1' 到 'sector5' 和 'mjsector' 的信息都连接到了一起。运行下方单元格中的代码来查看 sector 变量。.

In [ ]:
sector['sector']

你还能做些什么呢？如果你看看所有的 sector1 的类别，可以试试把它们结合起来，也许有用。例如，有几种类别都含有能源 ("Energy") 的字样。还有几种类别看起来和能源有关但是并不带能源 ("Energy") 的字样，比如热力 ("Thermal") 和 水力 ("Hydro")。一些类别使用可再生能源 "Renewable Energy" 的字样，所以你可能会建一个独立的可再生能源 "Renewable Energy" 类别。

类似的，一些类别包含交通 "Transportation" 字样，还有一些相关的类别，像是高速公路 "Highways"。

在下一个单元格中，找到所有的带有'Energy' 字样的 sector1 值。对于找到的每行数据，将字符串 'energy' 放到新的一列 'sector1_aggregates' 中。为交通 "Transportation" 做相同的操作。 

In [ ]:
import re

# Create the sector1_aggregates variable
sector.loc[:,'sector1_aggregates'] = sector['sector1']

# TODO: The code above created a new variable called sector1_aggregates. 
#       Currently, sector1_aggregates has all of the same values as sector1
#       For this task, find all the rows in sector1_aggregates with the term 'Energy' in them, 
#       For all of these rows, replace whatever is the value is with the term 'Energy'.
#       The idea is to simplify the category names by combining various categories together.
#       Then, do the same for the term 'Transportation
# HINT: You can use the contains() methods. See the documentation for how to ignore case using the re library
# HINT: You might get an error saying "cannot index with vector containing NA / NaN values." 
#       Try converting NaN values to something else like False or a string

print('Number of unique sectors after cleaning:', len(list(sector['sector1_aggregates'].unique())))

# 结语

使用 pandas 创建虚拟变量相对很简单；但是，通常你需要先清洗数据。